In [46]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv(r'C:\Users\Risha\OneDrive\Documents\OneDrive\Desktop\a\SongRecommender\songdata.csv.zip')

df.drop(columns=['link'],inplace=True)

df = df.drop_duplicates()

df['tag'] = df['artist']+" "+df['text']

newdf = df[['song','tag']]

corpus = []

lemm = WordNetLemmatizer()


for i in newdf['tag']:
    token = word_tokenize(i.lower())
    lemmatize = [lemm.lemmatize(tokens) for tokens in token]
    res = (" ".join(lemmatize))
    corpus.append(res)

import string
def pun(txt):
    return txt.translate(str.maketrans('','',string.punctuation))

corpus = [pun(text) for text in corpus]

newdf.loc[:, 'tag'] = corpus

vectorizer = TfidfVectorizer(stop_words='english')

x = vectorizer.fit_transform(newdf['tag'])



def rec(song_name, top_n=5):
    song_name = song_name.strip().lower()
    newdf['song_lower'] = newdf['song'].str.lower()

    if song_name not in newdf['song_lower'].values:
        return f"'{song_name}' not found in dataset."

    idx = newdf[newdf['song_lower'] == song_name].index[0]
    query_vec = x[idx]

    # ✅ Compute similarity only for this one song
    sim_scores = cosine_similarity(query_vec, x).flatten()
    sim_scores[idx] = -1  # exclude the song itself

    top_indices = sim_scores.argsort()[::-1][:top_n]
    recommended = [newdf.iloc[i]['song'] for i in top_indices]
    return recommended






C:\Users\Risha\AppData\Local\Temp\ipykernel_8680\1541283105.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf['song_lower'] = newdf['song'].str.lower()


"'unstopable' not found in dataset."

In [3]:
import nltk

# Download required NLTK corpora
nltk.download('wordnet')      # For WordNetLemmatizer
nltk.download('punkt')        # For word_tokenize
nltk.download('omw-1.4')      # Optional: multilingual WordNet support

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Risha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Risha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Risha\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
import joblib

# Save TF-IDF vectorizer
joblib.dump(vectorizer, 'vectorizer.joblib')

# Save TF-IDF matrix
joblib.dump(x, 'tfidf_matrix.joblib')

# Save cleaned DataFrame
joblib.dump(newdf, 'songdata.joblib')




['songdata.joblib']

In [48]:
vectorizer = joblib.load('vectorizer.joblib')
x = joblib.load('tfidf_matrix.joblib')
newdf= joblib.load('songdata.joblib')


In [ ]:
import streamlit as st

st.header('Song Recommendation')